In [1]:
! wget https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_large.pt
! pip install git+https://github.com/facebookresearch/sam2.git

--2024-10-12 10:59:53--  https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_large.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.164.78.121, 18.164.78.72, 18.164.78.81, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.164.78.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 898083611 (856M) [application/vnd.snesdev-page-table]
Saving to: 'sam2.1_hiera_large.pt'

sam2.1_hiera_large. 100%[===================>] 856.48M   273MB/s    in 3.5s    

2024-10-12 10:59:56 (245 MB/s) - 'sam2.1_hiera_large.pt' saved [898083611/898083611]

  Cloning https://github.com/facebookresearch/sam2.git to /tmp/pip-req-build-ndr6c5g9
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/sam2.git /tmp/pip-req-build-ndr6c5g9
  Resolved https://github.com/facebookresearch/sam2.git to commit c98aa6bea377d5c000cdc80197ce402dbf5304dc
  Installing build dependencies ... done
  Getting requireme

In [ ]:
import torch
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO

checkpoint = "sam2.1_hiera_large.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"
predictor = SAM2ImagePredictor(build_sam2(model_cfg, checkpoint))


image_url = 'https://plus.unsplash.com/premium_photo-1664298004972-af1ad3b52321?fm=jpg&q=60&w=3000&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8MXx8YnJvd24lMjBiZWFyfGVufDB8fDB8fHww'
response = requests.get(image_url)
image = Image.open(BytesIO(response.content))

input_point = np.array([[1997 // 2, 1500]])   
input_label = np.array([1]) 

with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    predictor.set_image(image)
    masks, _, _ = predictor.predict(input_point, input_label)